In [4]:
from random import randrange
from random import seed
from csv import reader
from math import sqrt
import numpy as np
import random

## Prepare dataset

In [2]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

filename = 'sonar.all-data.csv'
dataset = load_csv(filename)

# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

{'M': 0, 'R': 1}

## Shuffle, split training, test dataset

In [5]:
random.shuffle(dataset)

n_training = int(len(dataset) * 0.8)
training_set = dataset[:n_training]
test_set = dataset[n_training:]

print (len(training_set), len(test_set))

166 42


In [6]:
print(np.array(test_set)[:,-1])

[0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.]


## Build tree

In [7]:
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [8]:
def split(index, value, dataset):
	left, right = [], []
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [24]:
N_FEATURES = int(sqrt(len(training_set[1]) - 1))
MAX_DEPTH = 15
MIN_SIZE = 1

Each node contains the information of feature index, criterion value, gini score and lastly split groups (left and right) 

In [25]:
def make_node(training_set):
    class_values = [0.0, 1.0]
    
    selected_features = np.arange(N_FEATURES)
    np.random.shuffle(selected_features)
    
    selected_features = selected_features[:N_FEATURES]
    
    trained_feature_index = 1000
    trained_value = 1000.0
    trained_gini_score = 1000.0
    trained_group = None
    
    for feature_index in selected_features:
        for row in training_set:
            value = row[feature_index]
            group = split(feature_index, value, training_set)
            gini = gini_index(group, class_values)
            
            if gini < trained_gini_score:
                trained_feature_index = feature_index
                trained_value = value
                trained_gini_score = gini
                trained_group = group
                
    return {'trained_feature_index':trained_feature_index, 'trained_value':trained_value, 'trained_gini_score':trained_gini_score, 'trained_group':trained_group}           

In [26]:
def terminal_value(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

def expand_branch(node, depth):
    left, right = node['trained_group']

    if len(left) == 0 or len(right) == 0:
        value = terminal_value(left + right)
        node['left'] = value
        node['right'] = value
        return

    if depth >= MAX_DEPTH:
        node['left'], node['right'] = terminal_value(left), terminal_value(right)
        return

    if len(left) <= MIN_SIZE:
        node['left'] = terminal_value(left)
    else:
        node['left'] = make_node(left)
        expand_branch(node['left'], depth + 1)

    if len(right) <= MIN_SIZE:
        node['right'] = terminal_value(right)
    else:
        node['right'] = make_node(right)
        expand_branch(node['right'], depth + 1)
    

In [27]:
def make_tree(training_set):
    root = make_node(training_set)
    expand_branch(root, 1)
    return root

In [28]:
NUM_TREES = 10
trees = []
for i in range(NUM_TREES):
    trees.append(make_tree(training_set))

print(len(trees))

10


## Bagging result from each tree

In [29]:
def predict(node, row):
	if row[node['trained_feature_index']] < node['trained_value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [35]:
result = []
for row in test_set:
    predictions = [predict(tree, row) for tree in trees]
    #print(predictions)
    bagged_result = max(set(predictions), key=predictions.count)
    #print(bagged_result)
    result.append(bagged_result)

In [36]:
predicted_classes = np.array(result).astype(float)

actual_classes = np.array(test_set)[:,-1]

final_result = actual_classes == predicted_classes

print (sum(final_result) / len(final_result))

0.5238095238095238
